<a href="https://colab.research.google.com/github/louis-perdrix/Modal/blob/main/MTD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import numpy as np

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms import Lambda
from torchvision.transforms import Compose


training_data = datasets.FashionMNIST(".",download=True,train=True, transform=Compose([ToTensor(), torch.flatten]))
test_data = datasets.FashionMNIST(".",download=True,train=False, transform=Compose([ToTensor(), torch.flatten]))

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [ ]:
type(training_data[1][0])

torch.Tensor

In [ ]:
training_data[1][0].size()

torch.Size([784])

In [ ]:
28*28

784

On veut des tableaux unidimensionnels de taille 28*28 = 784

In [ ]:
training_data[0][1]

9

In [ ]:
model_TD1 = torch.nn.Sequential(torch.nn.Linear(784, 1))

In [ ]:
def trainer_float(dataset,model,loss_fn,epoch=10,batch_size=1,rate=1e-4):
  optimizer = torch.optim.Adam(model.parameters(), rate)
  data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)
  for i in tqdm(range(epoch)):
    for x in data_loader:
      y_pred = model(x[0])
      loss = loss_fn(y_pred,x[1].float)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()


In [ ]:
from tqdm.notebook import tqdm
              

In [ ]:
trainer_float(training_data, model_TD1, torch.nn.MSELoss())

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
def success_rate(model,test_data):
  count=0
  for test in test_data:
    if abs(model(test[0]) - test[1])<=0.5:
      count +=1
  return 100*count/len(test_data)


In [ ]:
len(test_data)

10000

In [ ]:
success_rate(model_TD1, test_data)

36.05

In [ ]:
model_10 = torch.nn.Sequential(torch.nn.Linear(784, 10), torch.nn.LogSoftmax())

In [ ]:
torch.nn.LogSoftmax()

LogSoftmax(dim=[0.1, 2])

In [ ]:
def trainer(dataset,model,loss_fn,epoch=10,batch_size=64,rate=1e-4):
  optimizer = torch.optim.Adam(model.parameters(), rate)
  data_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
  for i in tqdm(range(epoch)):
    for x in data_loader:
      y_pred = model(x[0])
      loss = loss_fn(y_pred,x[1])
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

In [ ]:
trainer(training_data, model_10, torch.nn.NLLLoss())

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
def success_rate_vector(model,test_data):
  count = 0
  for test in test_data:
    with torch.no_grad():
      if abs(torch.argmax(model.cuda()(test[0].cuda())) - test[1])==0:
        count += 1
  return 100*count/len(test_data)


In [ ]:
success_rate_vector(model_10, test_data)

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


84.85

Plus rapide et plus juste 

In [ ]:
model_activation = torch.nn.Sequential(torch.nn.Linear(784, 392), torch.nn.ReLU(), torch.nn.Linear(392, 128), torch.nn.ReLU(), torch.nn.Linear(128, 10), torch.nn.LogSoftmax())

In [ ]:
trainer(training_data, model_activation, torch.nn.NLLLoss())

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
success_rate_vector(model_activation, test_data)

86.9

ReLu est la meilleure 

In [ ]:
import cloudpickle as pickle
from google.colab import files

def torch_saver(net,file="temp"):
  with open(file,'wb') as f:
    pickle.dump(net,f)
    files.download(file)

In [ ]:
torch_saver(model_activation)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def trainer_GPU(dataset,model,loss_fn,epoch=10,batch_size=64,rate=1e-4):
  optimizer = torch.optim.Adam(model.parameters(), rate)
  data_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
  for i in tqdm(range(epoch)):
    for x in data_loader:
      y_pred = model.cuda()(x[0].cuda())
      loss = loss_fn(y_pred,x[1].cuda())
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

In [ ]:
trainer_GPU(training_data, model_activation, torch.nn.NLLLoss())

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [ ]:
success_rate_vector(model_activation, test_data)

88.48